# Enzo Sleep volumes lister
Get list of volumes per conditions/subject as an excel file, and generate a list of 1st-level covariates to regress out the volumes around the transition between concatenated sleep sequences (as we have to concatenate temporally disjoint sleep sequences to get enough volumes for analysis).
Before using this script, reorganize the data using the following command: `python pathmatcher.py -i "F:\Hyperc2019\sleep_fMRI_Enzo\sleep_enzo_fmri_extracted_epochsminlength28" -o "F:\Hyperc2019\sleep_fMRI_Enzo\sleep_enzo_reorg" -ri "(\dir)/(([^_]+).+)" -ro "\3/data/\1/func/\2" --copy`

by Stephen Karl Larroque, Coma Science Group - GIGA-Consciousness - University & Hospital of Liege
2019
v1.1
License: MIT

In [ ]:
# Forcefully autoreload all python modules
%load_ext autoreload
%autoreload 2

In [ ]:
import pathmatcher
import pandas as pd

In [ ]:
# PARAMETERS - EDIT ME
inputpath = r'F:\Hyperc2019\sleep_fMRI_Enzo\sleep_enzo_hyperc_allmanualdone'

In [ ]:
res = pathmatcher.main('-i "%s" -ri "(\\dir)/data/(\\dir)/func/[^_]+_(\\d+)_(\\d+)\.(?:img|nii)"' % inputpath, return_report=True, regroup=True)

In [ ]:
res

In [ ]:
newres = []
for subject, v in res[0].items():
    for condition, v2 in v.items():
        for session, v3 in v2.items():
            volumes = v3.keys()[0]
            filepath = v3.values()[0][0]
            newres.append({'subject': subject, 'condition': condition, 'session': session, 'volumes': volumes, 'filepath': filepath})
newres

In [ ]:
cf_res = pd.DataFrame(newres)
cf_res = cf_res[['subject', 'condition', 'session', 'volumes', 'filepath']]
cf_res

In [ ]:
if cf_res.to_excel('res.xls'):
    print('List of sessions/volumes per subject and condition saved in res.xls')
else:
    print('ERROR: could not save the list of sessions/volumes.')

----------------------

## Create 1st-level covariates to scrub volumes just after a transition between 2 concatenated epoch runs

In [ ]:
import matlab_wrapper
import os
import pathmatcher
import re
import sys
# start a Matlab session
mlab = matlab_wrapper.MatlabSession()
# add current folder to the path to have access to helper .m scripts, this needs to be done before each command call
mlab.workspace.addpath(os.path.abspath(''))
#mlab.workspace.cd(os.path.abspath(''))

In [ ]:
# PARAMETERS - EDIT ME
inputpath = r'F:\Hyperc2019\sleep_fMRI_Enzo\sleep_enzo_hyperc_allmanualdone'
volstoskip = 5  # number of volumes to skip after each transition to another acquisition set of epochs
matlabfileslistmode = 2  # choose between mode 1 (spm files listing as in preprocessing script) or mode 2 (regex_files as in conn loader scripts)


In [ ]:
res, [conflicts1, conflicts2] = pathmatcher.main('-i "%s" -ri "^\\dirnodot/data/\\dirnodot/func$" --dir' % inputpath, return_report=True, regroup=False)
res

In [ ]:
# Please copy this into a regex_files.m in the same folder as this script, else matlabfileslistmode == 2 won't work (only mode 1 will work, and only if you have SPM12 in your path)
# Also you need to restart the kernel AFTER you add this regex_files.m file in the same folder as this notebook.
mlistfilesfunc = r'''
function filesList = regex_files(dirpath, regex)
% filesList = regex_files(dirpath, regex)
% Extract files from a directory using regular expression

    % Get all files in directory
    filesList = dir(dirpath);
    % Filter out directories
    filesList = filesList(~[filesList.isdir]);
    % Use regular expression to filter only the files we want
    filesList = regexp({filesList.name}, regex, 'match');
    % Concatenate the filenames in a cellarray
    %filesList = {filesList.name};
    % Remove empty matches
    filesList = [filesList{:}];
    % Prepend the full path before each filename (so that we get absolute paths)
    if length(filesList) > 0
        filesList = cellfun(@(f) fullfile(dirpath, f), filesList, 'UniformOutput', false);
    end
    % Return directly the string instead of the cell array if there is only one file matched
    if length(filesList) == 1
        filesList = filesList{1};
    end
end
'''

In [ ]:
allvolslist = []

for p in res:
    ipath = os.path.join(inputpath, p[0])
    # Use Matlab to list the files, so that we ensure we get them in the same order as the preprocessing script we use
    mlab.put('dirpath', ipath)
    if matlabfileslistmode == 1:
        mlab.eval(r"[res] = cellstr(spm_select('FPList', dirpath, '^.+\.(img|nii)$'))")  # convert to cellstr the chararray output of the matlab function, else the wrapper mangles the output
    elif matlabfileslistmode == 2:
        mlab.eval(r"[res] = cellstr(regex_files(dirpath, '^.+\.(img|nii)$'))")  # convert to cellstr the chararray output of the matlab function, else the wrapper mangles the output
    else:
        raise Exception('Incorrect matlabfileslistmode supplied!')
    volslist = mlab.get('res').tolist()
    if isinstance(volslist, str):
        volslist = [volslist]
    # Extract the volumens count from the filename (it's the last digits) and convert to integers
    volslistint = [int(re.search(r'(\d+)\.(?:img|nii)', v).group(1)) for v in volslist]
    # Generate the 1st-level covariate (as many 0s or 1s as there are volumes, and we set 1 just after the transitions, else 0)
    covar = [0] * volslistint[0]
    if len(volslistint) > 1:
        for i in range(1, len(volslistint)):
            v = volslistint[i]
            if v > volstoskip:
                covar.extend(([1] * volstoskip) + ([0] * (v-volstoskip)))
            else:
                covar.extend([1] * v)
    # Save 1st-level covariate into a txt file for usage in CONN
    with open(os.path.join(ipath, 'epochtransitioncovar.txt'), 'wb') as f:
        f.write('\n'.join(str(x) for x in covar))
    # Store list of volumes, this allows to manually check if the order is alright
    allvolslist.extend(volslistint)

# Save list of all volumes for manual check
with open(os.path.join(inputpath, 'allvolslist.txt'), 'wb') as f:
    f.write('\n'.join(str(x) for x in allvolslist))

print('All done!')